# Examples of how to build concept-based explanations

Load the model and list modules to find where to split it.

In [ ]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("gpt2")

print(list(model.named_children()))
del model

### Split the model using the `ModelWithSplitPoints` class

In [2]:
from interpreto import ModelWithSplitPoints

splitted_model = ModelWithSplitPoints(
    model_or_repo_id="gpt2",
    split_points="transformer.h.1.mlp",
    model_autoclass=AutoModelForCausalLM,
)

### Load the dataset and compute activations

In [3]:
from datasets import load_dataset

rotten_tomatoes = load_dataset("cornell-movie-review-data/rotten_tomatoes")["train"]["text"]

activations = splitted_model.get_activations(
    rotten_tomatoes[10:], select_strategy=ModelWithSplitPoints.activation_strategies.TOKENS
)  # TODO: find why the kernel crashes
# TODO make a test (maybe scaling) that would catch this

print(activations.shape)

Using the latest cached version of the dataset since cornell-movie-review-data/rotten_tomatoes couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /home/antonin.poche/.cache/huggingface/datasets/cornell-movie-review-data___rotten_tomatoes/default/0.0.0/aa13bc287fa6fcab6daf52f0dfb9994269ffea28 (last modified on Fri May 16 17:50:46 2025).


: 

### Create and fit the concept explainer

In [ ]:
from interpreto.concepts import ICAConcepts

concept_explainer = ICAConcepts(splitted_model, nb_concepts=10)

concept_explainer.fit(activations)

### Interpret the concepts

In [15]:
from interpreto.concepts.interpretations import TopKInputs

some_words = [
    "excellent",
    "amazing",
    "fantastic",
    "outstanding",
    "great",
    "perfect",
    "wonderful",
    "superb",
    "love",
    "loved",
    "satisfied",
    "recommend",
    "best",
    "delightful",
    "pleasant",
    "enjoyed",
    "happy",
    "awesome",
    "brilliant",
    "incredible",
    "flawless",
    "impressive",
    "top-notch",
    "well-done",
    "five stars",
    "worth it",
    "will buy again",
    "high quality",
    "fast delivery",
    "friendlyterrible",
    "awful",
    "bad",
    "poor",
    "disappointing",
    "hate",
    "horrible",
    "worst",
    "boring",
    "rude",
    "unsatisfied",
    "never again",
    "waste",
    "problem",
    "issue",
    "slow",
    "cheap",
    "low quality",
    "broken",
    "not recommended",
    "annoying",
    "frustrating",
    "one star",
    "unacceptable",
    "defective",
    "dirty",
    "late delivery",
    "noisy",
    "difficult",
    "didn't work",
]

interpretations = concept_explainer.interpret(
    TopKInputs,
    concepts_indices="all",
    source=TopKInputs.sources.INPUTS,
    granularity=TopKInputs.granularities.WORD,
    inputs=some_words,
    k=5,
)

In [ ]:
for concept_id, words_importance in interpretations.items():
    print(f"Concept {concept_id}: {words_importance}")